# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J DB 환경 설정

In [4]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

True

In [5]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True,   # 향상된 스키마 사용 설정
)

In [6]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[#1271]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-445d1251-deab.production-orch-0068.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687))): ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다')
Transaction failed and will be retried in 1.1616255836245943s (Failed to read from defunct connection IPv4Address(('si-445d1251-deab.production-orch-0068.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687))))
[#1270]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687))): ConnectionAbortedError(10053, '현재 연결은 사용자의 호스트 시스템의 소프트웨어의 의해 중단되었습니다')
Unable to retrieve routing information
Transaction failed and will be retried in 1.7693798764985256s (Unable to retrieve routing information)


[{'Movie_Count': 4803}]

---

## 2. Neo4j 기반 질의 응답 시스템 (GraphVector RAG) 구현하기
- Neo4j에 저장된 임베딩 벡터를 활용한 문서 유사도 검색 방법을 살펴본다.

### 2.1 벡터 검색(Semantic Search)


#### 1) **Graph DB 초기화**

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

# Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
graph_vector = Neo4jVector.from_existing_index(
    embeddings,  # 사용할 임베딩 모델 지정
    url=os.getenv("NEO4J_URI"),  # Neo4j 데이터베이스 연결 URI (환경 변수에서 가져옴)
    username=os.getenv("NEO4J_USERNAME"),  # Neo4j 데이터베이스 사용자 이름
    password=os.getenv("NEO4J_PASSWORD"),  # Neo4j 데이터베이스 비밀번호
    index_name="movie_content_embeddings",  # 사용할 벡터 인덱스 이름 (이미 Neo4j에 생성되어 있어야 함)
    text_node_property="overview"  # 텍스트 검색 시 반환할 노드의 속성 (영화 개요)
)

#### 2) **벡터 검색 (유사도 기준)**

In [10]:
# 한국어로 된 자연어 쿼리를 사용하여 의미적으로 유사한 영화 검색
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."

# 유사도 검색 수행
similar_docs = graph_vector.similarity_search_with_score(
    query,
    k=5, # 유사도 상위 5개 문서 검색
    return_embeddings=False, # 임베딩 반환 안함 (결과 간소화)
)

# # 각 문서와 해당 유사도 점수를 함께 표시
# for doc, score in similar_docs:
#     print(f"줄거리: {doc.page_content[:100]}..., 유사도: {score}")
#     print("영화 제목:", doc.metadata.get("title"))
#     print("-" * 50)  

In [14]:
import pprint
similar_docs

[(Document(metadata={'rating': 6.2, 'runtime': 100, 'tagline': 'A Great Human Story... Makes A Mighty Motion Picture!!!', 'title': 'Sands of Iwo Jima', 'released': '1949-12-14'}, page_content="The relationship between Sergeant Stryker and a group of rebellious recruits is made difficult by the Sergeant's tough training tactics. At Tarawa, the leathernecks have a chance to see Stryker in action, and begin to appreciate him."),
  0.7099435329437256),
 (Document(metadata={'rating': 6.9, 'runtime': 126, 'tagline': 'A true story of survival... declassified.', 'title': 'Rescue Dawn', 'released': '2006-09-09'}, page_content="A US Fighter pilot's epic struggle of survival after being shot down on a mission over Laos during the Vietnam War."),
  0.7096529006958008),
 (Document(metadata={'rating': 8.3, 'runtime': 195, 'tagline': 'Whoever saves one life, saves the world entire.', 'title': "Schindler's List", 'released': '1993-11-29'}, page_content='The true story of how businessman Oskar Schindle

In [13]:
# 각 문서와 해당 유사도 점수를 함께 표시
for doc, score in similar_docs:
    print(f"줄거리: {doc.page_content[:100]}..., 유사도: {score}")
    print("영화 제목:", doc.metadata.get("title"))
    print("-" * 50)  

줄거리: The relationship between Sergeant Stryker and a group of rebellious recruits is made difficult by th..., 유사도: 0.7099435329437256
영화 제목: Sands of Iwo Jima
--------------------------------------------------
줄거리: A US Fighter pilot's epic struggle of survival after being shot down on a mission over Laos during t..., 유사도: 0.7096529006958008
영화 제목: Rescue Dawn
--------------------------------------------------
줄거리: The true story of how businessman Oskar Schindler saved over a thousand Jewish lives from the Nazis ..., 유사도: 0.7085556983947754
영화 제목: Schindler's List
--------------------------------------------------
줄거리: Wounded in Africa during World War II, Nazi Col. Claus von Stauffenberg returns to his native German..., 유사도: 0.7026262283325195
영화 제목: Valkyrie
--------------------------------------------------
줄거리: In a place where killers are celebrated as heroes, these filmmakers challenge unrepentant death-squa..., 유사도: 0.7017545700073242
영화 제목: The Act of Killing
----------------

In [15]:
# 메타데이터 출력
doc.metadata

{'rating': 7.5,
 'runtime': 115,
 'tagline': 'A story of killers who win, and the society they build.',
 'title': 'The Act of Killing',
 'released': '2012-08-31'}

#### 3) **MMR 알고리즘을 활용한 다양성 검색**
- MMR(Maximal Marginal Relevance) 알고리즘
- 질의와의 관련성 + 결과 간 중복 최소화(다양성)를 동시에 만족하는 문서 5개를 선택하는 과정
- 사용 하는 이유 
    - 상위 결과가 거의 동일한 내용의 문서들로 채워짐
    - RAG에서 → 컨텍스트 낭비, 추론 다양성 감소

In [16]:
# 벡터 검색 쿼리 실행
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."

similar_docs = graph_vector.max_marginal_relevance_search(
    query=query,
    k=5,
    fetch_k=20,  # 더 많은 후보를 가져옴
    lambda_mult=0.5,  # 0에 가까울수록 다양성 높음 (0.5는 적당한 균형을 의미함)
)

for doc in similar_docs:
    print(f"문서 내용: {doc.page_content[:100]}...")
    print("영화 제목:", doc.metadata.get("title"))
    print("-" * 50)  

문서 내용: The relationship between Sergeant Stryker and a group of rebellious recruits is made difficult by th...
영화 제목: Sands of Iwo Jima
--------------------------------------------------
문서 내용: Inspired by true events, this film takes place in Rwanda in the 1990s when more than a million Tutsi...
영화 제목: Hotel Rwanda
--------------------------------------------------
문서 내용: Wounded in Africa during World War II, Nazi Col. Claus von Stauffenberg returns to his native German...
영화 제목: Valkyrie
--------------------------------------------------
문서 내용: A post-apocalyptic tale, in which a lone man fights his way across America in order to protect a sac...
영화 제목: The Book of Eli
--------------------------------------------------
문서 내용: The Pianist is a film adapted from the biography of Wladyslaw Szpilman. A Jewish-Polish pianist who ...
영화 제목: The Pianist
--------------------------------------------------


#### 4) **그래프 경로를 활용한 검색 확장**

- 필모그래피(filmography) : 한 사람이 참여한 영화(또는 영상 작품)의 목록

In [ ]:
# Neo4j 그래프 객체 생성
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=True,
)

In [11]:
def find_movies_and_actors_filmography(query, graph_vector, k=5):
    """
    특정 영화에 출연한 배우들의 필모그래피를 찾는 함수
    
    Args:
        query: 검색 쿼리 (예: "2차 세계대전 영화")
        graph_db: 벡터 저장소 객체
        k: 검색할 영화 수
    
    Returns:
        영화와 배우 정보가 계층적으로 정리된 결과
    """
    # 1단계: 벡터 검색으로 영화 검색
    results = graph_vector.similarity_search(query, k=k)
    print(f"벡터스토어 검색된 영화 수: {len(results)}")
    
    # 검색된 영화 제목 추출
    movie_titles = []
    for doc in results:
        title = doc.metadata.get("title")
        if title:
            movie_titles.append(title)
    
    # 제목이 없으면 빈 결과 반환
    if not movie_titles:
        return {"movies": [], "message": "검색된 영화가 없습니다."}
    
 
    # 2단계: 영화와 출연 배우 정보 가져오기
    # Cypher 쿼리 설명:
    # 1. MATCH (movie:Movie): Movie 레이블을 가진 노드 찾기
    # 2. WHERE ANY(...): 검색된 영화 제목 중 하나라도 포함하는 영화 필터링
    # 3. OPTIONAL MATCH: 영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용
    # 4. collect(actor): 각 영화별로 출연 배우들을 배열로 수집
    # 5. RETURN: 영화 정보와 배우 정보를 함께 반환
    movies_actors_query = """
    // 영화 제목 중 하나라도 포함하는 영화 필터링
    MATCH (movie:Movie)
    WHERE ANY(title IN $movie_titles WHERE movie.title CONTAINS title)

    // 영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용
    OPTIONAL MATCH (movie)<-[acted:ACTED_IN]-(actor:Person)

    // 각 영화별로 출연 배우들을 배열로 수집
    WITH movie, collect(actor) as actors

    // 영화 정보와 배우 정보를 함께 반환
    RETURN 
        movie.title as movie_title,
        movie.released as release_date,
        movie.rating as rating,
        movie.overview as overview,
        [actor IN actors | actor.name] as actor_names,   // 배우 이름 배열
        [actor IN actors | {name: actor.name, id: id(actor)}] as actors  // 배우 정보 배열
    """
    
    # 영화와 출연 배우 정보 쿼리 실행
    # 벡터 db에서 검색한 movie_titles를 파라미터로 전달
    search_movies = graph.query(movies_actors_query, params={"movie_titles": movie_titles})
    
    # 배우 ID 목록 추출
    actor_ids = []
    for movie in search_movies:
        for actor in movie.get("actors", []):
            if "id" in actor:
                actor_ids.append(actor["id"])
    
    # 중복 제거 (동일한 배우가 여러 영화에 출연할 수 있음)
    actor_ids = list(set(actor_ids))
    
    # 배우가 없으면 영화 정보만 반환
    if not actor_ids:
        return {
            "movies": search_movies,
            "message": "검색된 영화에서 배우 정보를 찾을 수 없습니다."
        }
    
    # 3단계: 각 배우의 필모그래피 가져오기
    # Cypher 쿼리 설명:
    # 1. MATCH (actor:Person): 특정 ID를 가진 배우 노드 찾기
    # 2. WHERE id(actor) IN $actor_ids: 앞서 찾은 배우 ID 목록으로 필터링
    # 3. MATCH (actor)-[:ACTED_IN]->(movie:Movie): 배우가 출연한 영화 찾기
    # 4. WHERE NOT movie.title IN $movie_titles: 원래 검색된 영화는 제외 (중복 방지)
    # 5. WITH actor.name, collect(...): 배우별로 출연 영화 정보 수집
    # 6. RETURN: 배우 이름과 출연 영화 목록 반환
    # 7. ORDER BY: 배우 이름 알파벳 순으로 정렬
    actor_filmography_query = """
    // 특정 ID를 가진 배우 노드 찾기
    MATCH (actor:Person)
    WHERE id(actor) IN $actor_ids
    
    // 배우가 출연한 영화 찾기
    MATCH (actor)-[:ACTED_IN]->(movie:Movie)
    
    // 원래 검색된 영화는 제외 (중복 방지)
    WHERE NOT movie.title IN $movie_titles
    
    WITH actor.name as actor_name, collect({
        title: movie.title,
        released: movie.released,
        rating: movie.rating
    }) as other_movies  // 배우별로 출연 영화 정보 수집
    RETURN actor_name, other_movies     // 배우 이름과 출연 영화 목록 반환
    ORDER BY actor_name  // 배우 이름 알파벳 순으로 정렬
    """
    
    actor_filmographies = graph.query(
        actor_filmography_query, 
        params={"actor_ids": actor_ids, "movie_titles": movie_titles}
    )
    
    # 결과 데이터 구조화 - 영화 정보와 배우 필모그래피를 함께 반환
    result = {
        "search_movies": search_movies,
        "actor_filmographies": actor_filmographies
    }
    
    return result

- 검색 쿼리 실행하기

In [12]:
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."
search_results = find_movies_and_actors_filmography(
    query=query,
    graph_vector=graph_vector,
    k=3  # 상위 3개 영화만 검색
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 19, column: 51, offset: 605} for query: '\n    // 영화 제목 중 하나라도 포함하는 영화 필터링\n    MATCH (movie:Movie)\n    WHERE ANY(title IN $movie_titles WHERE movie.title CONTAINS title)\n\n    // 영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용\n    OPTIONAL MATCH (movie)<-[acted:ACTED_IN]-(actor:Person)\n\n    // 각 영화별로 출연 배우들을 배열로 수집\n    WITH movie, collect(actor) as actors\n\n    // 영화 정보와 배우 정보를 함께 반환\n    RETURN \n        movie.title as movie_title,\n        movie.released as release_date,\n        movie.rating as rating,\n        movie.overview as overview,\n        [actor IN actors | actor.name] as actor_names,   // 배우 이름 배열\n   

벡터스토어 검색된 영화 수: 3


In [13]:
# 결과 출력 - 영화 정보
for movie in search_results["search_movies"]:
    print(f"\n제목: {movie['movie_title']} ({movie.get('release_date', '연도 정보 없음')})")
    print(f"평점: {movie.get('rating', '평점 정보 없음')}")
    print(f"출연 배우: {', '.join(movie.get('actor_names', []))}")
    if movie.get('overview'):
        print(f"줄거리: {movie['overview'][:150]}..." if len(movie['overview']) > 150 else movie['overview'])
    print("-" * 50)


제목: Rescue Dawn (2006-09-09)
평점: 6.9
출연 배우: Steve Zahn, Christian Bale, Marshall Bell, Toby Huss, Pat Healy
A US Fighter pilot's epic struggle of survival after being shot down on a mission over Laos during the Vietnam War.
--------------------------------------------------

제목: Valkyrie (2008-12-25)
평점: 6.7
출연 배우: Terence Stamp, Tom Cruise, Bill Nighy, Carice van Houten, Kenneth Branagh
줄거리: Wounded in Africa during World War II, Nazi Col. Claus von Stauffenberg returns to his native Germany and joins the Resistance in a daring plan to cre...
--------------------------------------------------

제목: The Act of Killing (2012-08-31)
평점: 7.5
출연 배우: Anwar Congo, Herman Koto, Syamsul Arifin, Ibrahim Sinik, Yapto Soerjosoemarno
줄거리: In a place where killers are celebrated as heroes, these filmmakers challenge unrepentant death-squad leaders to dramatize their role in genocide. The...
--------------------------------------------------


In [14]:
# 배우별 필모그래피 출력
print("\n\n=== 배우별 출연 영화 목록 ===")
for actor in search_results["actor_filmographies"]:
    print(f"\n배우: {actor['actor_name']}")
    print("출연 영화:")
    for idx, movie in enumerate(actor.get('other_movies', []), 1):
        print(f"  {idx}. {movie['title']} ({movie.get('released', '연도 정보 없음')}) - "
              f"평점: {movie.get('rating', '평점 정보 없음')}")
    print("-" * 50)



=== 배우별 출연 영화 목록 ===

배우: Bill Nighy
출연 영화:
  1. The Constant Gardener (2005-08-31) - 평점: 6.8
  2. Pirates of the Caribbean: Dead Man's Chest (2006-06-20) - 평점: 7.0
  3. Flushed Away (2006-10-22) - 평점: 6.0
  4. Notes on a Scandal (2006-12-25) - 평점: 6.9
  5. Underworld: Rise of the Lycans (2009-01-22) - 평점: 6.2
  6. The Boat That Rocked (2009-04-01) - 평점: 7.2
  7. Astro Boy (2009-10-15) - 평점: 6.1
  8. Wild Target (2010-06-17) - 평점: 6.4
  9. Arthur Christmas (2011-02-22) - 평점: 6.7
  10. Rango (2011-03-02) - 평점: 6.6
  11. The Best Exotic Marigold Hotel (2011-11-28) - 평점: 6.9
  12. About Time (2013-08-16) - 평점: 7.8
  13. I, Frankenstein (2014-01-22) - 평점: 5.0
  14. The Second Best Exotic Marigold Hotel (2015-02-26) - 평점: 6.3
--------------------------------------------------

배우: Carice van Houten
출연 영화:
  1. Black Book (2006-09-01) - 평점: 7.2
  2. Repo Men (2010-03-18) - 평점: 6.2
  3. Race (2016-02-19) - 평점: 7.1
--------------------------------------------------

배우: Christian Bale
출연 영화:

### 2.2 **LCEL** 사용하여 **RAG** 체인을 구성


#### 1) **Graph DB 기반 Retriever 활용**

- Neo4jVector의 저장된 임베딩과 유사도 검색 

In [15]:
retriever = graph_vector.as_retriever(
    search_kwargs={"k":5},  # 검색할 문서 수
)

query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."
results = retriever.invoke(query)

for result in results:
    print(result.page_content[:100])  # 첫 100자만 출력
    # print(result.metadata)  # 영화 제목 출력
    print(result.metadata['title'])  # 영화 제목 출력
    print("-" * 50)  

A US Fighter pilot's epic struggle of survival after being shot down on a mission over Laos during t
Rescue Dawn
--------------------------------------------------
Wounded in Africa during World War II, Nazi Col. Claus von Stauffenberg returns to his native German
Valkyrie
--------------------------------------------------
In a place where killers are celebrated as heroes, these filmmakers challenge unrepentant death-squa
The Act of Killing
--------------------------------------------------
In Nazi-occupied France during World War II, a group of Jewish-American soldiers known as "The Baste
Inglourious Basterds
--------------------------------------------------
The Earth is attacked by unknown forces. As people everywhere watch the world's great cities fall, L
Battle: Los Angeles
--------------------------------------------------


- Neo4jVector 기반 RAG chain 예시

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def format_movie_info(movie):
    """
    영화 정보를 문자열로 포맷팅하는 함수
    """

    return f"""
    영화 제목: {movie.get('title', '정보 없음')}
    개봉일: {movie.get('released', '정보 없음')}
    평점: {movie.get('rating', '정보 없음')}
    줄거리: {movie.get('overview', '줄거리 정보 없음')}
    """

def format_results(docs):
    """
    검색된 문서에 있는 영화 정보를 포맷팅하는 함수
    """
    formatted_results = []
    for doc in docs:
        movie_info = doc.metadata
        movie_info['overview'] = doc.page_content   # 줄거리 추가
        formatted_results.append(format_movie_info(movie_info))   # 포맷팅된 영화 정보 추가
    
    return "\n".join(formatted_results)

# RAG용 Retriever 객체 생성
retriever = graph_vector.as_retriever(
    search_kwargs={"k":5},  # 검색할 문서 수
)

# LLM 객체 생성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

# Prompt 템플릿 정의
template = '''당신은 영화 추천 전문가로서 오직 주어진 정보에 기반하여 객관적이고 정확한 답변을 제공합니다.

[주어진 영화 정보]
{context}

[질문]
{question}

# 답변 작성 지침:
1. 제공된 영화 정보에 명시된 사실만 사용하세요.
2. 간결하고 정확하게 답변하세요.
3. 제공된 정보에 없는 내용은 "제공된 정보에서 해당 내용을 찾을 수 없습니다"라고 답하세요.
4. 영화의 제목, 평점 등 주요 정보를 포함해서 답변하세요.
5. 한국어로 자연스럽고 이해하기 쉽게 답변하세요.
'''

# Prompt 객체 생성
prompt = ChatPromptTemplate.from_template(template)

# context 생성 단계(질문 → 검색 → 포맷팅)
# - retriever 결과를 사람이 읽을 수 있는 context로 변환
build_context = retriever | RunnableLambda(format_results)

# - 템플릿에 증강하기 위해 데이터 구성 단계
rag_input = {
    "context": build_context,
    "question": RunnablePassthrough()   # query 질문을 그대로 전달
}

# RAG chain 파이프라인  : 질문 -> 검색 -> Prompt 증강 → LLM → Output 
rag_chain = (
    rag_input | prompt | llm | StrOutputParser()
)

# Chain 실행
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."
answer = rag_chain.invoke(query)
print(answer)

2차 세계대전을 배경으로 군인들의 활약상을 그린 영화는 다음과 같습니다.

- 영화 제목: Valkyrie  
  개봉일: 2008-12-25  
  평점: 6.7  
  줄거리: 아프리카에서 부상당한 나치 대령 클라우스 폰 슈타우펜베르크가 독일로 돌아와 저항군에 합류하고, 히틀러 암살과 그림자 정부 수립을 위한 대담한 계획에 참여하는 이야기입니다.

- 영화 제목: Inglourious Basterds  
  개봉일: 2009-08-18  
  평점: 7.9  
  줄거리: 나치 점령 하의 프랑스에서 유대계 미국인 병사들이 나치를 공포에 몰아넣기 위해 잔혹한 작전을 수행하며, 파리의 영화관을 운영하는 프랑스계 유대인 소녀와 교차하는 이야기입니다.


#### 2) **Graph DB 자체 검색 함수 활용**

In [18]:
# 그래프 DB 검색 기능을 활용한 RAG 체인 예시

# 검색 함수 정의
def graph_search_retriever(query):
    # 유사도 검색 수행
    similar_docs = graph_vector.similarity_search_with_score(
        query,
        k=5, # 유사도 상위 5개 문서 검색
        return_embeddings=False, # 임베딩 반환 안함 (결과 간소화)
    )
    
    # 검색 결과 포맷팅
    results = []
    for doc, score in similar_docs:
        movie_info = doc.metadata.copy()
        movie_info['overview'] = doc.page_content
        movie_info['search_score'] = score
        results.append(movie_info)
    
    return results

# 결과 포맷팅 함수
def format_graph_results(results):
    formatted_results = []
    
    for movie_info in results:
        formatted_results.append(format_movie_info(movie_info))
    
    return "\n".join(formatted_results)

# 그래프 DB 기반 RAG 체인 구성
rag_input = {
        "context": graph_search_retriever | RunnableLambda(format_graph_results),
        "question": RunnablePassthrough()
    }

graph_rag_chain =  rag_input | prompt | llm | StrOutputParser()

# 체인 실행
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."
graph_answer = graph_rag_chain.invoke(query)

print(graph_answer)

2차 세계대전을 배경으로 군인들의 활약상을 그린 영화는 다음과 같습니다.

- 영화 제목: Valkyrie  
  개봉일: 2008-12-25  
  평점: 6.7  
  줄거리: 아프리카에서 부상당한 나치 대령 클라우스 폰 슈타우펜베르크가 독일로 돌아와 저항군에 합류해 히틀러 암살과 그림자 정부 수립을 계획하며 쿠데타를 주도하는 이야기입니다.

- 영화 제목: Inglourious Basterds  
  개봉일: 2009-08-18  
  평점: 7.9  
  줄거리: 나치 점령 하의 프랑스에서 유대계 미국인 병사들이 나치를 공포에 떨게 하기 위해 잔혹한 작전을 수행하고, 파리의 영화관을 운영하는 프랑스계 유대인 소녀와 만나면서 벌어지는 이야기입니다.
